# Metrics

This notebook is a rough first draft for metric creation.

In [ ]:
import cv2
from pathlib import Path
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
data_directory = Path('../data/images/floorplan/output/')
files = [x for x in data_directory.glob('*.png')]

In [ ]:
from fmlwright.core.preprocessing import image_to_geodataframe
import networkx as nx

# Neigbour regions

In [ ]:
def find_neighbouring_areas(gdf):
    buffer = 2 # add small buffer to ensure overlap.

    matching_ids = {}
    for i, row in gdf.iterrows():
        matching_id = gdf[~gdf.geometry.buffer(buffer).disjoint(row.geometry)].index
        matching_id = [x for x in matching_id if i != x]
        matching_ids[i] = matching_id
    
    return matching_ids

In [ ]:
def create_graph(matching_ids, gdf):
    G = nx.Graph()
    edges = []
    for key, val in matching_ids.items():
        for _v in val:
            edges.append((key, _v))
    
    pos = {}
    for i, row in gdf.iterrows():
        pos[i] = [row.geometry.centroid.x, row.geometry.centroid.y]
        
    G.add_edges_from(edges)
    for x in G.nodes():
        coords = pos[x]
        G.nodes[x]['x'] = coords[0]
        G.nodes[x]['y'] = coords[1]
    return G

def get_coords(G):
    coords = {}
    for x in G.nodes(data=True):
        coords[x[0]] = [x[1]['x'], x[1]['y']]
    return coords

In [ ]:
files

In [ ]:
img = cv2.imread(str(files[11]))

In [ ]:
connectivity = image_to_geodataframe(img)
results = find_neighbouring_areas(connectivity)
G = create_graph(results, connectivity)

In [ ]:
fig, ax = plt.subplots()
plt.imshow(img)
nx.draw(G, get_coords(G))

---
# Bar plot

In [ ]:
import pandas as pd

In [ ]:
connectivity.plot(color=connectivity['colors'])

In [ ]:
areas = connectivity[['category', 'colors']].copy()
areas['area'] = connectivity.geometry.area

In [ ]:
areas

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
areas[['area']].T.plot.bar(ax=ax, stacked=True, color=areas['colors'], legend=False)
plt.axis("off");

----
# Skeletonize

In [ ]:
from skimage.morphology import skeletonize

In [ ]:
res = skeletonize(img)

In [ ]:
plt.imshow(img)

In [ ]:
plt.imshow(res)